In [38]:
%%capture
# Install deps in current jupyter kernel python
import sys
!{sys.executable} -m pip install google-cloud-storage
!{sys.executable} -m pip install google-cloud-bigquery

In [87]:
# Pull in script deps
import requests
import json
import csv
import datetime
from google.cloud import storage
from google.cloud import bigquery
from google.cloud.bigquery import LoadJobConfig
from google.cloud.bigquery import SchemaField

In [88]:
# globals, constants, etc.
TZ = 'EST04EDT'
BUCKET = 'bsc-caserta-datatest'
NOW = datetime.datetime.now()
YEAR = NOW.year
MONTH = NOW.month
DAY = NOW.day
HOUR = NOW.hour
MIN = NOW.minute
SEC = NOW.second
BQ = {
  'dataset_name': 'coinmarketcap',
  'table_name': '{}{}{}'.format(YEAR, MONTH, DAY)
}
FILENAME = '{}/{}/{}'.format(YEAR, MONTH, DAY)

def upload_blob(bucket_name, source_file_name, destination_blob_name):
  """Uploads a file to the bucket."""
  storage_client = storage.Client()
  bucket = storage_client.get_bucket(bucket_name)
  blob = bucket.blob(destination_blob_name)

  blob.upload_from_filename(source_file_name)

def get_blob(bucket_name, source_file_name):
  client = storage.Client()
  bucket = client.get_bucket(bucket_name)
  blob = bucket.get_blob(source_file_name)
  return blob.download_as_string()


In [103]:
# Get the data from the API, run through the JSON output,
# package it into python lists, and dump a CSV file locally
r = requests.get('https://api.coinmarketcap.com/v2/ticker/?start=0&limit=1') #grab metadata
num_cryptocurrencies = r.json()[u'metadata']['num_cryptocurrencies']

csv_data = [['id', 'name', 'symbol', 'usd_price',
            'usd_market_cap', 'usd_pct_chg_1h', 'usd_pct_chg_24h',
            'usd_pct_chg_7d', 'usd_volume_24h', 'total_supply',
            'max_supply', 'circulating_supply']]

for i in range(0, num_cryptocurrencies, 100):
  r = requests.get('https://api.coinmarketcap.com/v2/ticker/?start={}&limit=100'.format(i))
  data = r.json()[u'data']

  for d in set(data):
    x = data[d]
    x_quote = x['quotes']['USD']

    csv_data.append([x['id'], x['name'], x['symbol'], x_quote['price'],
          x_quote['market_cap'], x_quote['percent_change_1h'], x_quote['percent_change_24h'],
          x_quote['percent_change_7d'], x_quote['volume_24h'], x['total_supply'],
          x['max_supply'], x['circulating_supply']])

# quick sanity check of data
if len(csv_data)-1 != num_cryptocurrencies:
  print(len(csv_data)-1)
  print(num_cryptocurrencies)
  
print(csv_data[0:5])
  
# write everything to a local CSV file
with open('{}.csv'.format(DAY), 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, dialect='excel')
    for i in csv_data:
      csvwriter.writerow(i)

2091
2090
[['id', 'name', 'symbol', 'usd_price', 'usd_market_cap', 'usd_pct_chg_1h', 'usd_pct_chg_24h', 'usd_pct_chg_7d', 'usd_volume_24h', 'total_supply', 'max_supply', 'circulating_supply'], [131, 'Dash', 'DASH', 163.584386234, 1371752838.0, -0.13, -0.63, -8.55, 170713367.683366, 8385598.0, 18900000.0, 8385598.0], [1896, '0x', 'ZRX', 0.9905214863, 536426475.0, -2.34, 34.61, 33.05, 122315800.553535, 1000000000.0, None, 541559655.0], [1042, 'Siacoin', 'SC', 0.006690731, 250403123.0, 0.21, 3.08, -6.9, 3600825.45931643, 37425376014.0, None, 37425376014.0], [2083, 'Bitcoin Gold', 'BTG', 26.7405630008, 462392430.0, 0.3, 0.98, 1.74, 2501787.85944407, 17391799.0, 21000000.0, 17291799.0]]


In [104]:
# Put the CSV file in the bucket
upload_blob(BUCKET, '{}.csv'.format(DAY), FILENAME)

In [106]:
# Load CSV in BigQuery
bq = bigquery.Client()

SCHEMA = [
  SchemaField('id', 'INTEGER', mode='required'),
  SchemaField('name', 'STRING', mode='required'),
  SchemaField('symbol', 'STRING', mode='required'),
  SchemaField('usd_price', 'FLOAT', mode='required'),
  SchemaField('usd_market_cap', 'FLOAT'),
  SchemaField('usd_pct_chg_1h', 'FLOAT'),
  SchemaField('usd_pct_chg_24h', 'FLOAT'),
  SchemaField('usd_pct_chg_7d', 'FLOAT'),
  SchemaField('usd_volume_24h', 'FLOAT'),
  SchemaField('total_supply', 'FLOAT'),
  SchemaField('max_supply', 'FLOAT'),
  SchemaField('circulating_supply', 'FLOAT'),
]

dataset_ref = bq.dataset(BQ['dataset_name'])

load_config = bigquery.LoadJobConfig()
load_config.skip_leading_rows = 1
load_config.schema = SCHEMA

uri = 'gs://{}/{}/{}/{}'.format(BUCKET, YEAR, MONTH, DAY)
bq.delete_table(dataset_ref.table(BQ['table_name'])) 
load_job = bq.load_table_from_uri(
    uri,
    dataset_ref.table(BQ['table_name']),
    job_config=load_config)

print('Starting job {}'.format(load_job.job_id))

load_job.result()  # Waits for table load to complete.
print('Job finished.')

destination_table = bq.get_table(dataset_ref.table(BQ['table_name']))
print('Loaded {} rows.'.format(destination_table.num_rows))

Starting job 74b8ae84-4b16-41ab-820b-aaea203f9baf
Job finished.
Loaded 2091 rows.


In [117]:
#Run some queries
# 1 -- price > $8000

q = ('SELECT name, symbol, usd_price FROM `coinmarketcap.20181017` WHERE usd_price > 8000;')
query_job = bq.query(q)  # API request
rows = query_job.result()  # Waits for query to finish

i = 0
for row in rows:
  i += 1
  print(row)
  
print('\n{} coins have a price > $8000'.format(i))

Row(('Bit20', 'BTWTY', 215104.182628), {'name': 0, 'symbol': 1, 'usd_price': 2})
Row(('Project-X', 'NANOX', 47656.0145424), {'name': 0, 'symbol': 1, 'usd_price': 2})
Row(('42-coin', '42', 22073.4112929), {'name': 0, 'symbol': 1, 'usd_price': 2})

3 coins have a price > $8000


In [118]:
# 2 -- sum of top 100 market cap

q = ('SELECT name, symbol, usd_market_cap FROM `coinmarketcap.20181017` ORDER BY usd_market_cap DESC LIMIT 100;')
query_job = bq.query(q)  # API request
rows = query_job.result()  # Waits for query to finish

total = 0
for row in rows:
  total += row[2]
  
print('Total market cap of top 100 coins is ${}'.format(total))

Total market cap of top 100 coins is $204192023116.0


In [113]:
# 3 -- supply < $5M

q = ('SELECT name, symbol, ((COALESCE(total_supply, max_supply) - circulating_supply) * usd_price) AS usd_avail_supply FROM `coinmarketcap.20181017` WHERE COALESCE(total_supply, max_supply) IS NOT NULL AND (COALESCE(total_supply, max_supply) * usd_price) < 5000000 ORDER BY usd_avail_supply DESC;')
query_job = bq.query(q)  # API request
rows = query_job.result()  # Waits for query to finish

for row in rows:
  print(row)

Row(('KUN', 'KUN', 4381628.2564508), {'name': 0, 'symbol': 1, 'usd_avail_supply': 2})
Row(('NEVERDIE', 'NDC', 4155834.0687454203), {'name': 0, 'symbol': 1, 'usd_avail_supply': 2})
Row(('Traceability Chain', 'TAC', 3870131.7095113457), {'name': 0, 'symbol': 1, 'usd_avail_supply': 2})
Row(('Sharpay', 'S', 3683062.2059170282), {'name': 0, 'symbol': 1, 'usd_avail_supply': 2})
Row(('Kryll', 'KRL', 3345861.327841599), {'name': 0, 'symbol': 1, 'usd_avail_supply': 2})
Row(('Bethereum', 'BETHER', 3316780.3479705425), {'name': 0, 'symbol': 1, 'usd_avail_supply': 2})
Row(('GoldMint', 'MNTP', 3265332.750980303), {'name': 0, 'symbol': 1, 'usd_avail_supply': 2})
Row(('Simmitri', 'SIM', 3194086.8583279857), {'name': 0, 'symbol': 1, 'usd_avail_supply': 2})
Row(('SkinCoin', 'SKIN', 3191072.04), {'name': 0, 'symbol': 1, 'usd_avail_supply': 2})
Row(('BlockMesh', 'BMH', 3143719.6861048345), {'name': 0, 'symbol': 1, 'usd_avail_supply': 2})
Row(('MTC Mesh Network', 'MTC', 3088311.375849707), {'name': 0, 'sy

In [114]:
# 4 -- 5 coins w/ greatest %age growth in last week

q = ('SELECT name, symbol, usd_pct_chg_7d FROM `coinmarketcap.20181017` WHERE usd_pct_chg_7d IS NOT NULL AND usd_pct_chg_7d > 0 ORDER BY usd_pct_chg_7d DESC LIMIT 5;')
query_job = bq.query(q)  # API request
rows = query_job.result()  # Waits for query to finish

for row in rows:
  print(row)

Row(('Cashcoin', 'CASH', 791.11), {'name': 0, 'symbol': 1, 'usd_pct_chg_7d': 2})
Row(('TRONCLASSIC', 'TRXC', 570.34), {'name': 0, 'symbol': 1, 'usd_pct_chg_7d': 2})
Row(('PitisCoin', 'PTS', 445.87), {'name': 0, 'symbol': 1, 'usd_pct_chg_7d': 2})
Row(('SuperCoin', 'SUPER', 442.05), {'name': 0, 'symbol': 1, 'usd_pct_chg_7d': 2})
Row(('FREE Coin', 'FREE', 435.57), {'name': 0, 'symbol': 1, 'usd_pct_chg_7d': 2})


In [119]:
# 5 -- how many tickers contain 'X'

q = ('SELECT symbol FROM `coinmarketcap.20181017` WHERE REGEXP_CONTAINS(symbol,"X");')
query_job = bq.query(q)  # API request
rows = query_job.result()  # Waits for query to finish

i = 0
for row in rows:
  i += 1
  
print('{} ticker symbols contain the letter "X"'.format(i))

271 ticker symbols contain the letter "X"
